In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libudev.so.1'

2023-07-04 04:05:52.977147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Fetch the jet tagging dataset from Open ML

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

np.save('X_test.npy', x_test)
np.save('y_test.npy', y_test)

## Construct a model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras import QConv2D
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.layers import Flatten, MaxPooling2D, Input

In [4]:
model = Sequential()
model.add(
    Input(shape=(28, 28, 1))
)

# Conv 1
model.add(
    QConv2D(
        16,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding='valid',
        name='conv1',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu1')
)
model.add(
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid', name='pool1')
)

# Conv 2
model.add(
    QConv2D(
        4,
        kernel_size=(5, 5),
        strides=(1, 1),
        padding='valid',
        name='conv2',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)
model.add(
    QActivation(activation=quantized_relu(4), name='relu2')
)
model.add(
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid', name='pool2')
)

# Flatten
model.add(Flatten())

# FC 1
model.add(
    QDense(
        30,
        name='fc1',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)

# FC 2
model.add(
    QDense(
        10,
        name='fc2',
        kernel_quantizer=quantized_bits(8, 4, alpha=1),
        bias_quantizer=quantized_bits(8, 4, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001)
    )
)

model.add(Activation(activation='softmax', name='softmax'))

model.summary()

2023-07-04 04:05:55.713287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (QConv2D)             (None, 24, 24, 16)        416       
                                                                 
 relu1 (QActivation)         (None, 24, 24, 16)        0         
                                                                 
 pool1 (MaxPooling2D)        (None, 12, 12, 16)        0         
                                                                 
 conv2 (QConv2D)             (None, 8, 8, 4)           1604      
                                                                 
 relu2 (QActivation)         (None, 8, 8, 4)           0         
                                                                 
 pool2 (MaxPooling2D)        (None, 4, 4, 4)           0         
                                                                 
 flatten (Flatten)           (None, 64)                0

## Train sparse

In [5]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

## Train the model

In [8]:
train = True
if train:
    adam = Adam(learning_rate=0.0001)
    model.compile(
        optimizer=adam, 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    model.fit(
        x_train,
        y_train,
        batch_size=1024,
        epochs=30,
        validation_data=(x_test, y_test),
        shuffle=True,
    )
    model = strip_pruning(model)
    model.save('LeNet_Prune_Model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('model_3/KERAS_check_best_model.h5', custom_objects=co)

Epoch 1/30
59/59 [==============================] - 9s 129ms/step - loss: 0.4486 - accuracy: 0.9059 - val_loss: 0.4232 - val_accuracy: 0.9105
Epoch 2/30
59/59 [==============================] - 7s 123ms/step - loss: 0.4276 - accuracy: 0.9080 - val_loss: 0.4005 - val_accuracy: 0.9170
Epoch 3/30
59/59 [==============================] - 7s 123ms/step - loss: 0.4089 - accuracy: 0.9116 - val_loss: 0.3809 - val_accuracy: 0.9191
Epoch 4/30
59/59 [==============================] - 7s 125ms/step - loss: 0.3909 - accuracy: 0.9153 - val_loss: 0.3790 - val_accuracy: 0.9159
Epoch 5/30
59/59 [==============================] - 7s 124ms/step - loss: 0.3812 - accuracy: 0.9169 - val_loss: 0.3682 - val_accuracy: 0.9202
Epoch 6/30
59/59 [==============================] - 7s 124ms/step - loss: 0.3698 - accuracy: 0.9196 - val_loss: 0.3465 - val_accuracy: 0.9263
Epoch 7/30
59/59 [==============================] - 7s 124ms/step - loss: 0.3560 - accuracy: 0.9225 - val_loss: 0.3355 - val_accuracy: 0.9253
Epoch 